In [76]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [77]:
class AE20B108_Q4:
  def __init__(self, num_balls, num_matches):
    self.__num_actions = 6
    self.__num_balls = num_balls
    self.__num_matches = num_matches
    self.__total_wicket = 0
    self.__match_ball = 0
    self.__alpha = 3.5
    self.__wicket = 4
    self.__M_X = 1
    self.__M_R = 6
    self.__prev_action = None
    self.__arm_pulls = np.zeros(self.__num_actions)
    self.__reward = np.zeros(self.__num_actions)
    self.__cost = np.zeros(self.__num_actions)
    self.__V_R = np.zeros(self.__num_actions)
    self.__V_X = np.zeros(self.__num_actions)
    self.__n = 0

  def get_action(self, wicket, runs_scored):
      if not self.__prev_action is None:
          self.__arm_pulls[self.__prev_action] += 1
          self.__reward[self.__prev_action] += runs_scored
          self.__cost[self.__prev_action] += wicket + self.__wicket/self.__num_balls
          self.__V_R[self.__prev_action] += runs_scored ** 2
          self.__V_X[self.__prev_action] += (wicket + self.__wicket/self.__num_balls) ** 2
      if self.__n < self.__num_actions:
          action = self.__n
      else:
          r_mean = self.__reward / self.__arm_pulls
          X_mean = (self.__cost + 1e-15) / self.__arm_pulls
          R_var = self.__V_R / self.__arm_pulls - (self.__reward / self.__arm_pulls) ** 2
          X_var = self.__V_X / self.__arm_pulls - ((self.__cost) / self.__arm_pulls) ** 2
          eps = np.sqrt(2 * R_var * np.log(self.__n ** self.__alpha) / self.__arm_pulls) + 3 * self.__M_R * np.log(self.__n ** self.__alpha) / self.__arm_pulls
          eta = np.sqrt(2 * X_var * np.log(self.__n ** self.__alpha) / self.__arm_pulls) + 3 * self.__M_X * np.log(self.__n ** self.__alpha) / self.__arm_pulls
          rho_hat = r_mean / X_mean
          c_hat = 1.4 * (eps + r_mean * eta) / X_mean
          action = np.argmax(rho_hat + c_hat)
      self.__n += 1
      self.__prev_action = action
      return action

In [78]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__p_out =np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_run =np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__wickets_left = 4
    self.__wicket = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__batting_order = np.array([0,1,2,3])

  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket, self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun = self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__wickets_left = 4
    self.__runs_scored = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__run_time = 0
    self.__start_time = 0
    self.__end_time = 0

    for ball in range(self.num_balls):
      if (self.__wickets_left > 0 ) :
        action = self.__get_action()
        self.__wicket, self.__runs_scored   = self.__get_outcome(action)
        self.__total_runs     = self.__total_runs + self.__runs_scored
        if (self.__wicket > 0 ):
          self.__wickets_left = self.__wickets_left -1
        self.__total_wickets  = self.__total_wickets + self.__wicket
        if (self.__wickets_left == 0):
          self.__get_action()
    return self.__total_runs, self.__total_wickets, self.__run_time


In [91]:
num_matches = 10
num_balls = 60
agent = AE20B108_Q4(num_balls,num_matches)
environment = Environment(100,agent)
score = np.zeros((num_matches,1))
run_time = np.zeros((num_matches,1))
wicket = np.zeros((num_matches,1))
for i in range(num_matches):
  score[i],wicket[i],run_time[i] = environment.innings()

In [92]:
score

array([[104.],
       [103.],
       [150.],
       [ 76.],
       [141.],
       [119.],
       [ 77.],
       [174.],
       [ 48.],
       [133.]])

In [93]:
np.sum(score)

1125.0